<a href="https://colab.research.google.com/github/OmdenaAI/omdena-colombia-socialnetwork/blob/task-5-visualization/src/tasks/task-5-visualization/carlos_bar_chart_race.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bar_chart_race -q

     |████████████████████████████████| 156 kB 2.7 MB/s 


In [2]:
import pandas as pd 
import bar_chart_race as bcr
import numpy as np

In [33]:
df = pd.read_csv("Corrected_final_all_with_sentiment.csv", index_col="Unnamed: 0", encoding="UTF-8")
df["hashtags"] = df["hashtags"].astype(str)
df["preprocessed_created_at"] = pd.to_datetime(df["preprocessed_created_at"])
sentiments = {
    0:"negative",
    1:"positive"}
df["sentiment_score"] = df["sentiment_score"].map(sentiments)
df_es = df[df["lang"] == "es"]
df_en = df[df["lang"] == "en"]
df.head()

,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,sentiment_score
0,2021-03-27T04:09:42+00:00,1.380000e+18,1.380000e+18,@Diputado_Canelo Hagamos otro por el uno de ma...,es,False,False,0.0,1.0,0.0,0.0,NaN,NaN,NaN,9.094314e+08,carmeno1635,🌻🌺Carmen Ochoa...🌺🌻,False,1.380000e+18,1.290000e+18,#1mayo,NaN,NaN,"[{""url"": ""https://pbs.twimg.com/media/ExdUpC3X...",2021-03-27,"['hacer', 'mayo', 'cazar', 'fantasma', 'mayo']",[''],[':/'],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['hacer', 'mayo', 'cazar', 'fantasma']",positive
1,2021-03-22T21:12:09+00:00,1.370000e+18,1.370000e+18,Después de esperar con ancias el #28F ahora es...,es,False,False,1.0,4.0,0.0,0.0,NaN,NaN,NaN,1.260000e+18,olga73427503,olga Lidia 👄💄🌹,False,NaN,NaN,"#28F , #1Mayo , #15Septiembre","nayibbukele , waraujo64",NaN,[],2021-03-22,"['despues', 'esperar', 'ancia', 'ahora', 'espe...",['💙🤍💙'],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['despues', 'esperar', 'ancia', 'ahora', 'espe...",positive
2,2021-03-22T12:30:53+00:00,1.370000e+18,1.370000e+18,Espero que ésto llegue hasta oídos de la nueva...,es,False,False,0.0,1.0,1.0,0.0,NaN,NaN,NaN,1.350000e+18,Mr_DonNadie,Dios del Olimpo 🇸🇻⚖️ UES,False,NaN,NaN,"#1Mayo , #BecasUniversitarias",AsambleaSV,NaN,[],2021-03-22,"['esperar', 'llegar', 'oido', 'nuevo', 'inicia...",[''],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['esperar', 'llegar', 'oido', 'nuevo', 'inicia...",negative
3,2021-04-04T12:56:55+00:00,1.380000e+18,1.380000e+18,A menos de un mes del #1Mayo Urkullu teme perd...,es,False,False,3.0,5.0,0.0,0.0,NaN,NaN,NaN,1.662306e+08,Antifaxismoa,Sare Antifaxista,False,NaN,NaN,"#1Mayo , #pandemia , #oasisvasco , #regimenjel...",NaN,NaN,[],2021-04-04,"['menos', 'mes', 'mayo', 'urkullu', 'temer', '...",[''],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['menos', 'mes', 'urkullu', 'temer', 'perder',...",negative
4,2021-04-03T20:14:57+00:00,1.380000e+18,1.380000e+18,La X Edición del Festival Internacional Un Pue...,es,False,False,1.0,3.0,0.0,1.0,NaN,NaN,NaN,1.320000e+18,Radiometrohaba1,Radio Metropolitana Cuba,False,NaN,NaN,"#LaHabana , #1Mayo , #MiHabanaEsMúsica , #Cuba...","AneliPupo , BenitezCordovi , LoureiroMilan , n...",https://t.co/caGq26zE1K,"[{""url"": ""https://pbs.twimg.com/media/EyE0s-AW...",2021-04-03,"['edicion', 'festival', 'internacional', 'puen...",[''],"[':/', ':/']",Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['edicion', 'festival', 'internacional', 'puen...",positive


In [4]:
df_en.head()

,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,sentiment_score
16690,2021-04-23T07:14:36+00:00,1.390000e+18,1.390000e+18,📢Join the seminar aiming at supporting #1May #...,en,False,False,5.0,4.0,0.0,1.0,NaN,NaN,NaN,1.050000e+18,_EuropeanForum,European Forum,False,NaN,NaN,"#1May , #1maggio , #1Mayo","LeftHeinz , Yolanda_Diaz_ , VisentiniLuca , lu...",NaN,"[{""url"": ""https://pbs.twimg.com/media/EzpAFBfX...",2021-04-23,"['join', 'seminar', 'aim', 'support', 'may', '...",['📢'],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['join', 'seminar', 'aim', 'support', 'trade',...",1
16691,2021-04-22T04:12:43+00:00,1.390000e+18,1.380000e+18,@julioacosta1701 @DiazCanelB @AlejandroGilF @r...,es,False,False,7.0,1.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.190000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","julioacosta1701 , DiazCanelB , AlejandroGilF ,...",NaN,[],2021-04-22,"['caliente', 'tweetazo', 'prevenirporlavida', ...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['caliente', 'tweetazo']",0
16692,2021-04-22T04:12:00+00:00,1.390000e+18,1.380000e+18,@julioacosta1701 @AliRubioGlez @Vicente7397772...,es,False,False,7.0,1.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.190000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","julioacosta1701 , AliRubioGlez , Vicente739777...",NaN,[],2021-04-22,"['sumo', 'prevenirporlavida', 'patriaomuerte',...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,['sumo'],0
16693,2021-04-22T04:09:48+00:00,1.390000e+18,1.380000e+18,@AdrinMartnezGo1 @julioacosta1701 @DiazCanelB ...,es,False,False,6.0,0.0,0.0,0.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.390000e+18,1.140000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","AdrinMartnezGo1 , julioacosta1701 , DiazCanelB...",NaN,[],2021-04-22,"['hay', 'diocancancan', 'prevenirporlavida', '...",['😅😂🤣🤣'],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['hay', 'diocancancan']",0
16694,2021-04-22T03:32:26+00:00,1.390000e+18,1.380000e+18,@VilmaPrez14 @Laura53538740 @Granma_Digital @D...,es,False,False,5.0,1.0,0.0,1.0,NaN,NaN,NaN,1.210000e+18,BauzaCuba,Alejandro Rodríguez Bauza,False,1.380000e+18,1.100000e+18,"#PrevenirPorLaVida , #PatriaOMuerte , #UnBlock...","VilmaPrez14 , Laura53538740 , Granma_Digital ,...",NaN,[],2021-04-22,"['con', 'mucha', 'conciencia', 'prevenirporlav...",[],[],Mayo_ENGLISH_tweets_stweet.csv,Twitter,en,NaN,NaN,NaN,"['con', 'mucha', 'conciencia']",1


In [5]:
df_es.head()

,created_at,id_str,conversation_id_str,full_text,twitter_lang,favorited,retweeted,retweet_count,favorite_count,reply_count,quote_count,quoted_status_id_str,quoted_status_short_url,quoted_status_expand_url,user_id_str,user_name,user_full_name,user_verified,in_reply_to_status_id_str,in_reply_to_user_id_str,hashtags,mentions,urls,media,preprocessed_created_at,preprocessed_data,emoji_list,emoticons_list,filename,data_source,lang,score,langTb,lang_langdetect,preprocessed_data_without_hashtags,sentiment_score
0,2021-03-27T04:09:42+00:00,1.380000e+18,1.380000e+18,@Diputado_Canelo Hagamos otro por el uno de ma...,es,False,False,0.0,1.0,0.0,0.0,NaN,NaN,NaN,9.094314e+08,carmeno1635,🌻🌺Carmen Ochoa...🌺🌻,False,1.380000e+18,1.290000e+18,#1mayo,NaN,NaN,"[{""url"": ""https://pbs.twimg.com/media/ExdUpC3X...",2021-03-27,"['hacer', 'mayo', 'cazar', 'fantasma', 'mayo']",[''],[':/'],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['hacer', 'mayo', 'cazar', 'fantasma']",1
1,2021-03-22T21:12:09+00:00,1.370000e+18,1.370000e+18,Después de esperar con ancias el #28F ahora es...,es,False,False,1.0,4.0,0.0,0.0,NaN,NaN,NaN,1.260000e+18,olga73427503,olga Lidia 👄💄🌹,False,NaN,NaN,"#28F , #1Mayo , #15Septiembre","nayibbukele , waraujo64",NaN,[],2021-03-22,"['despues', 'esperar', 'ancia', 'ahora', 'espe...",['💙🤍💙'],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['despues', 'esperar', 'ancia', 'ahora', 'espe...",1
2,2021-03-22T12:30:53+00:00,1.370000e+18,1.370000e+18,Espero que ésto llegue hasta oídos de la nueva...,es,False,False,0.0,1.0,1.0,0.0,NaN,NaN,NaN,1.350000e+18,Mr_DonNadie,Dios del Olimpo 🇸🇻⚖️ UES,False,NaN,NaN,"#1Mayo , #BecasUniversitarias",AsambleaSV,NaN,[],2021-03-22,"['esperar', 'llegar', 'oido', 'nuevo', 'inicia...",[''],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['esperar', 'llegar', 'oido', 'nuevo', 'inicia...",0
3,2021-04-04T12:56:55+00:00,1.380000e+18,1.380000e+18,A menos de un mes del #1Mayo Urkullu teme perd...,es,False,False,3.0,5.0,0.0,0.0,NaN,NaN,NaN,1.662306e+08,Antifaxismoa,Sare Antifaxista,False,NaN,NaN,"#1Mayo , #pandemia , #oasisvasco , #regimenjel...",NaN,NaN,[],2021-04-04,"['menos', 'mes', 'mayo', 'urkullu', 'temer', '...",[''],[],Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['menos', 'mes', 'urkullu', 'temer', 'perder',...",0
4,2021-04-03T20:14:57+00:00,1.380000e+18,1.380000e+18,La X Edición del Festival Internacional Un Pue...,es,False,False,1.0,3.0,0.0,1.0,NaN,NaN,NaN,1.320000e+18,Radiometrohaba1,Radio Metropolitana Cuba,False,NaN,NaN,"#LaHabana , #1Mayo , #MiHabanaEsMúsica , #Cuba...","AneliPupo , BenitezCordovi , LoureiroMilan , n...",https://t.co/caGq26zE1K,"[{""url"": ""https://pbs.twimg.com/media/EyE0s-AW...",2021-04-03,"['edicion', 'festival', 'internacional', 'puen...",[''],"[':/', ':/']",Mayo_SPANISH_tweets_stweet.csv,Twitter,es,NaN,NaN,NaN,"['edicion', 'festival', 'internacional', 'puen...",1


# Hashtags

In [6]:
def get_hashtag_counts(data): 
  counts_list = []
  for date in data["preprocessed_created_at"].unique():
    counts = {}
    counts["date"] = date
    sub_df = data[data["preprocessed_created_at"] == date]
    for _, row in sub_df.iterrows():
      for hashtag in row["hashtags"].split(","):
        hashtag = hashtag.replace("#","").strip()
        counts[hashtag] = counts.get(hashtag, 0) + 1
    counts_list.append(counts)
  return pd.DataFrame(counts_list)
hashtag_counts_es = get_hashtag_counts(df_es)
hashtag_counts_en = get_hashtag_counts(df_en)

In [7]:
hashtag_counts_es.set_index("date", inplace=True)
hashtag_counts_en.set_index("date", inplace=True)

hashtag_counts_es.sort_index(inplace=True)
hashtag_counts_en.sort_index(inplace=True)

hashtag_counts_es.fillna(0, inplace=True)
hashtag_counts_en.fillna(0, inplace=True)

hashtag_counts_es = hashtag_counts_es.astype(int)
hashtag_counts_en = hashtag_counts_en.astype(int)

hashtag_cumsum_es = hashtag_counts_es.cumsum(axis=0)
hashtag_cumsum_en = hashtag_counts_en.cumsum(axis=0)

del hashtag_cumsum_es["nan"]
del hashtag_cumsum_en["nan"]


In [8]:
hashtag_cumsum_es.to_csv("cumsum_es.csv")
hashtag_cumsum_en.to_csv("cumsum_en.csv")

In [9]:
hashtag_cumsum_es.head()

,1mayo,Colombia,AHORA,USA,Guerrillas,Culpables,FalsosPositivos,Venezuela,URGENTE,colombiaresiste,HOY,FRONTAL,ElPeorGobiernoDeLaHistoria,Ecuador,LeninMoreno,Octubre2019,ComisionVerdadYJusticia,ParoNacional,MariaPaulaRomo,SEPE,mentira,ingresominimo,PARONACIONAL,PeorGobiernoDeLaHistoria,Atención,DuqueInepto,PetroPresidente2020,Matarife,ColombiaDespreciaAUribe,UribeEsUnNarcoAsesino,PrimeraLínea,ElPeorCongresoDeLaHistoria,Destrucción,PrimeraLinea,EstallidoSinGloria,18OVerguenzaMundial,NoMasEstallidoDelictual,FueraFiscalChong,FueraAbbottEncunbridor,LimpiezaAlPoderJudicial,...,ETESEX,stopviolenciamachista,ContraElTerrorismoMachista,NacionalizacionServiciosIgualdad,rebelion,vil,Dinero,sucio,NoAlVotoDeConfianza,AlProfesorGaviriaLeDigo,congelarsalarios,paronacional26a,tokio2020,caidedo,robledo,calentamientoglobal,caricaturacolombia,NoGreenPass,TiraniaGlobal,niunomas,enfermosrenales,deudainhumana,lasaludesloprimero,BienvenidoSeptiembre,paguen,BASTA,ElMachismoMata,lasalud,100PrimerosDiasJuntos,NoAlBitcoin,PorUnMundoMasBonito,PorUnMejorFuturo,Cubadebate,LeidyCardenas,saqueo,caracoltv,UnaMentiraEs,uribedeberesponder,uribecobarde,NoMasOspina
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-03-22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2021-03-23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0
2021-03-24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0
2021-03-25,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,1,2,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0
2021-03-26,0,24,8,0,0,0,0,0,0,0,8,0,0,0,0,0,0,6,0,0,0,0,0,0,0,3,3,3,4,3,33,0,1,9,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0


In [10]:
hashtag_cumsum_en.head()

,1May,1maggio,1Mayo,28A,BisingPH,NoALaReformaTributaria,SOSColombia,ParoNacional28A,PrevenirPorLaVida,PatriaOMuerte,UnBlockCuba,SoñarYContinuar,Venceremos,TuEresElPresente,LeadersClimateSummit,Chile,ChileDesperto,PrimeraLinea,UNFCCC,UNSC,DDHH,PeaceAgreement,SOSColombiaDDHH,colombiansocialleaders,NoAlGlifosato,fracking,NuevasCiudadanias,Medellin,NarcoGoverment,8voCongresoPCC,CubaSocialista,SomosContinuidad,EducadoresCamagueyanos,Cuba,UnidosHacemosCuba,UnidosVenceremos,CubaViva,SomosCuba,nosestanmatando,NosEstánMatando,...,Atlántico,SeguimosEnResistencia,Dignidad,ProtestarNOEsUnDelito,StandWithHongKong,WhatsHappeningInThailand,ColombianSocialLeaders,ColombiaAmaAUribe,colombiadespreciaauribe,SemanaSanta2021,UribeALaCPI,uribista,CoalicionDeLaEsperanza,DeFajardoPienso,PapaFrancisco,graffart,bombingscience,chrome,comeback,fresh,funky,home,instagram,new,saturday,snapchat,SubscribeNow,YoParoPor,30Mar,SputnikVContraCovid,coventry2021,cityofculture2021,igerscoventry,circolombia,Violence,HelpPlease,International,Impunity,Murder,urgentmeasures
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-03-22,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
2021-03-23,0,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
2021-03-24,0,0,0,1,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
2021-03-26,0,0,0,1,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1
2021-03-27,0,0,0,1,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1


In [11]:
def get_top_n_hashtags(data, n = 50):
  row = data.loc[data.index.max()]
  row = row.to_dict()
  hashtags_sorted = sorted(row.items(), key = lambda x: x[1], reverse = True)
  dict_hashtags = dict(hashtags_sorted[:n])
  return dict_hashtags.keys()

In [ ]:
bcr.bar_chart_race(df=hashtag_cumsum_es[get_top_n_hashtags(hashtag_cumsum_es, 10)], figsize=(10.5,10), title="Hashtag counts for spanish tweets")

In [ ]:
bcr.bar_chart_race(df=hashtag_cumsum_en[get_top_n_hashtags(hashtag_cumsum_en, 10)], figsize=(10.5,10), title="Hashtag counts for english tweets")

# Sentiment

In [54]:
sentiments_all = df[["preprocessed_created_at", "sentiment_score"]].groupby(by=["preprocessed_created_at", "sentiment_score"]).size().reset_index()
sentiments_all = pd.pivot_table(sentiments_all, values=0, index=['preprocessed_created_at'],
                    columns=['sentiment_score'], aggfunc=np.sum).reset_index()
sentiments_all.set_index("preprocessed_created_at", inplace=True)
sentiments_all.fillna(0,inplace=True)
sentiments_all.sort_index(inplace=True)
sentiments_all = sentiments_all.astype(int)
sentiments_cumsum_all = sentiments_all.cumsum(axis=0)
sentiments_cumsum_all

sentiment_score,negative,positive
preprocessed_created_at,,
2021-03-22,2,5
2021-03-23,6,5
2021-03-24,10,5
2021-03-25,24,9
2021-03-26,87,22
...,...,...
2021-08-31,16507,5580
2021-09-01,16523,5583
2021-09-02,16539,5587


In [55]:
sentiments_es = df[["preprocessed_created_at", "sentiment_score"]].groupby(by=["preprocessed_created_at", "sentiment_score"]).size().reset_index()
sentiments_es = pd.pivot_table(sentiments_es, values=0, index=['preprocessed_created_at'],
                    columns=['sentiment_score'], aggfunc=np.sum).reset_index()
sentiments_es.set_index("preprocessed_created_at", inplace=True)
sentiments_es.fillna(0,inplace=True)
sentiments_es.sort_index(inplace=True)
sentiments_es = sentiments_es.astype(int)
sentiments_cumsum_es = sentiments_es.cumsum(axis=0)
sentiments_cumsum_es

sentiment_score,negative,positive
preprocessed_created_at,,
2021-03-22,2,5
2021-03-23,6,5
2021-03-24,10,5
2021-03-25,24,9
2021-03-26,87,22
...,...,...
2021-08-31,16507,5580
2021-09-01,16523,5583
2021-09-02,16539,5587


In [56]:
sentiments_en = df[["preprocessed_created_at", "sentiment_score"]].groupby(by=["preprocessed_created_at", "sentiment_score"]).size().reset_index()
sentiments_en = pd.pivot_table(sentiments_en, values=0, index=['preprocessed_created_at'],
                    columns=['sentiment_score'], aggfunc=np.sum).reset_index()
sentiments_en.set_index("preprocessed_created_at", inplace=True)
sentiments_en.fillna(0,inplace=True)
sentiments_en.sort_index(inplace=True)
sentiments_en = sentiments_en.astype(int)
sentiments_cumsum_en = sentiments_en.cumsum(axis=0)
sentiments_cumsum_en

sentiment_score,negative,positive
preprocessed_created_at,,
2021-03-22,2,5
2021-03-23,6,5
2021-03-24,10,5
2021-03-25,24,9
2021-03-26,87,22
...,...,...
2021-08-31,16507,5580
2021-09-01,16523,5583
2021-09-02,16539,5587


In [57]:
bcr.bar_chart_race(df=sentiments_cumsum_all, figsize=(10.5,10), title="Sentiment counts for all tweets")

In [ ]:
bcr.bar_chart_race(df=sentiments_cumsum_en, figsize=(10.5,10), title="Sentiment counts for english tweets")

In [ ]:
bcr.bar_chart_race(df=sentiments_cumsum_es, figsize=(10.5,10), title="Sentiment counts for spanish tweets")